# Biomaker CA: Eruption configuration

This colab shows how to run experiments on the Eruption configuration. Eruption contains LAVA and FIRE as extra materials. See the code base for examples on how to create your own materials.

Set soil_unbalance_limit to 1/3 for a more optimal set of experiments. The value of 0 reproduces experiments in the [original YouTube video](https://youtu.be/e8Gl0Ns4XiM).

Copyright 2023 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

## Imports

In [ ]:
#@title install selforg package
# install the package locally
!pip install --upgrade -e git+https://github.com/google-research/self-organising-systems.git#egg=self_organising_systems&subdirectory=biomakerca
# activate the locally installed package (otherwise a runtime restart is required)
import pkg_resources
import importlib
# Reload the resources because we uninstalled and reinstalled some packages.
importlib.reload(pkg_resources)
pkg_resources.get_distribution("self_organising_systems").activate()

In [ ]:
#@title imports & notebook utilities

import cv2
import numpy as np
import jax.random as jr
import jax.numpy as jp
from jax import vmap
from jax import jit
import jax
import time

from self_organising_systems.biomakerca import environments as evm
from self_organising_systems.biomakerca.agent_logic import BasicAgentLogic
from self_organising_systems.biomakerca.agent_logic import adapt_dna_to_different_basic_logic
from self_organising_systems.biomakerca.mutators import BasicMutator
from self_organising_systems.biomakerca.mutators import RandomlyAdaptiveMutator
from self_organising_systems.biomakerca.step_maker import step_env
from self_organising_systems.biomakerca.display_utils import zoom, imshow, add_text_to_img
from self_organising_systems.biomakerca.custom_ipython_display import display
from self_organising_systems.biomakerca.env_logic import ReproduceOp
from self_organising_systems.biomakerca.env_logic import env_perform_one_reproduce_op
from self_organising_systems.biomakerca.env_logic import process_structural_integrity_n_times
from self_organising_systems.biomakerca.extensions.eruption import EruptionTypeDef
from self_organising_systems.biomakerca.extensions.eruption import make_eruption_excl_fs
from self_organising_systems.biomakerca.extensions.eruption import create_eruption_env
from self_organising_systems.biomakerca.extensions.eruption import run_eruption_env
from self_organising_systems.biomakerca.extensions.eruption import test_freq_lava
from self_organising_systems.biomakerca.extensions.eruption import test_wave_lava
from self_organising_systems.biomakerca.extensions.eruption import update_slice_with_lava
from self_organising_systems.biomakerca.extensions.eruption import get_eruption_config

from self_organising_systems.biomakerca.utils import load_dna, save_dna
from self_organising_systems.biomakerca.environments import DefaultTypeDef
from self_organising_systems.biomakerca.env_logic import make_empty_upd_state
from self_organising_systems.biomakerca import cells_logic
from self_organising_systems.biomakerca import env_logic

tiny_dna = load_dna('persistence_1697720583799231923.npy')

import tqdm
import mediapy as media
from functools import partial
from IPython.display import clear_output
import matplotlib.pyplot as plt

## Create the eruption config and etd

In [ ]:
soil_unbalance_limit = 0 #@param [0, "1/3"] {type:"raw"}

config = get_eruption_config()
config.soil_unbalance_limit = soil_unbalance_limit
etd = config.etd
# Create the exclusive fs
excl_fs = make_eruption_excl_fs(etd)

print(config)

## Port old trained DNA to a DNA that contains more env types

In [ ]:
agent_model = "extended"
agent_logic = BasicAgentLogic(config, minimal_net=agent_model=="minimal")

mutator_type = "randomly_adaptive" #@param ['basic', 'randomly_adaptive']
sd = 1e-3
mutator = (BasicMutator(sd=sd, change_perc=0.2) if mutator_type == "basic"
           else RandomlyAdaptiveMutator(init_sd=sd, change_perc=0.2))

old_env_and_config = evm.get_env_and_config(
    "persistence", width_type="landscape", h=16)
old_config = old_env_and_config[1]
old_logic = BasicAgentLogic(old_config, minimal_net=agent_model == "minimal")

updated_tiny_dna = adapt_dna_to_different_basic_logic(
    tiny_dna, old_logic, agent_logic)
print(updated_tiny_dna.shape)

## Lava endurance tests

### init, sd=1e-4

In [ ]:
key = jr.PRNGKey(42)
h = 72
w = evm.infer_width(h, "landscape")
env = evm.create_default_environment(config, h, w)
env = evm.place_seed(env, w//2, config)

sd = 1e-4
mutator = RandomlyAdaptiveMutator(init_sd=sd, change_perc=0.2)

# enhance the dna with mutator params.
key, ku = jr.split(key)
init_dna = agent_logic.initialize(ku)
key, ku = jr.split(key)
init_dna = mutator.initialize(ku, init_dna)

test_freq_lava(key, env, config, init_dna, agent_logic, mutator)
test_wave_lava(key, env, config, init_dna, agent_logic, mutator)

### init, sd=1e-3

In [ ]:
key = jr.PRNGKey(42)
h = 72
w = evm.infer_width(h, "landscape")
env = evm.create_default_environment(config, h, w)
env = evm.place_seed(env, w//2, config)

sd = 1e-3
mutator = RandomlyAdaptiveMutator(init_sd=sd, change_perc=0.2)

# enhance the dna with mutator params.
key, ku = jr.split(key)
init_dna = agent_logic.initialize(ku)
key, ku = jr.split(key)
init_dna = mutator.initialize(ku, init_dna)

test_freq_lava(key, env, config, init_dna, agent_logic, mutator)
test_wave_lava(key, env, config, init_dna, agent_logic, mutator)

### tinydna, big world, 1e-4

In [ ]:
key = jr.PRNGKey(42)
h = 72
w = evm.infer_width(h, "landscape")
env = evm.create_default_environment(config, h, w)
env = evm.place_seed(env, w//2, config)

sd = 1e-4
mutator = RandomlyAdaptiveMutator(init_sd=sd, change_perc=0.2)
# enhance the dna with mutator params.
key, ku = jr.split(key)
init_dna = mutator.initialize(ku, updated_tiny_dna)

zoom_sz = 8
test_freq_lava(key, env, config, init_dna, agent_logic, mutator, zoom_sz=zoom_sz)
test_wave_lava(key, env, config, init_dna, agent_logic, mutator, zoom_sz=zoom_sz)

### tinydna, big world, sd=1e-3

In [ ]:
key = jr.PRNGKey(42)
h = 72
w = evm.infer_width(h, "landscape")
env = evm.create_default_environment(config, h, w)
env = evm.place_seed(env, w//2, config)

sd = 1e-3
mutator = RandomlyAdaptiveMutator(init_sd=sd, change_perc=0.2)
# enhance the dna with mutator params.
key, ku = jr.split(key)
init_dna = mutator.initialize(ku, updated_tiny_dna)

zoom_sz = 8
test_freq_lava(key, env, config, init_dna, agent_logic, mutator, zoom_sz=zoom_sz)
test_wave_lava(key, env, config, init_dna, agent_logic, mutator, zoom_sz=zoom_sz)

### Smaller worlds, 1e-4

In [ ]:
key = jr.PRNGKey(42)
h = 36
w = evm.infer_width(h, "landscape")
env = evm.create_default_environment(config, h, w)
env = evm.place_seed(env, w//2, config)

sd = 1e-4
mutator = RandomlyAdaptiveMutator(init_sd=sd, change_perc=0.2)
# enhance the dna with mutator params.
key, ku = jr.split(key)
init_dna = mutator.initialize(ku, updated_tiny_dna)

zoom_sz = 16
test_freq_lava(key, env, config, init_dna, agent_logic, mutator, zoom_sz=zoom_sz)
test_wave_lava(key, env, config, init_dna, agent_logic, mutator, zoom_sz=zoom_sz)

### small, sd=1e-3

In [ ]:
key = jr.PRNGKey(42)
h = 36
w = evm.infer_width(h, "landscape")
env = evm.create_default_environment(config, h, w)
env = evm.place_seed(env, w//2, config)

sd = 1e-3
mutator = RandomlyAdaptiveMutator(init_sd=sd, change_perc=0.2)
# enhance the dna with mutator params.
key, ku = jr.split(key)
init_dna = mutator.initialize(ku, updated_tiny_dna)

zoom_sz = 16
test_freq_lava(key, env, config, init_dna, agent_logic, mutator, zoom_sz=zoom_sz)
test_wave_lava(key, env, config, init_dna, agent_logic, mutator, zoom_sz=zoom_sz)

## Large, diverse environment
Center: nothing

Left: waves of lots of lava. note that a wave arrives as a random event now, not a predetermined frequency.

Right: constant little lava

In [ ]:
agent_model = "extended" #@param ['minimal', 'extended']
agent_logic = BasicAgentLogic(config, minimal_net=agent_model=="minimal")

mutator_type = "randomly_adaptive" #@param ['basic', 'randomly_adaptive']
sd = 1e-3 #@param ['1e-3', '1e-4'] {type:"raw"}
mutator = (BasicMutator(sd=sd, change_perc=0.2) if mutator_type == "basic"
           else RandomlyAdaptiveMutator(init_sd=sd, change_perc=0.2))

### Testing init on a regular height (72)

In [ ]:
# first just initialize the environment to a certain state, without making videos.
key = jr.PRNGKey(47)

h = 72

n_steps = 50000

use_tiny_dna = False

if use_tiny_dna:
  program = updated_tiny_dna
else:
  ku, key = jr.split(key)
  program = agent_logic.initialize(ku)

ku, key = jr.split(key)
program = mutator.initialize(ku, program)

# 128 is TOO SMALL!
N_MAX_PROGRAMS = 256

programs = jp.repeat(program[None, :], N_MAX_PROGRAMS, axis=0)

env = create_eruption_env(h, config)
ku, key = jr.split(key)
programs, env = run_eruption_env(
    ku, config, programs, env, agent_logic, mutator, n_steps, zoom_sz=6)


In [ ]:
# continue...
n_steps = 50000
ku, key = jr.split(key)
programs, env = run_eruption_env(
    ku, config, programs, env, agent_logic, mutator, n_steps, zoom_sz=6,
    steps_per_frame=64, when_to_double_speed=[])

### Testing tiny_dna on a small height (36)

In [ ]:
# first just initialize the environment to a certain state, without making videos.
key = jr.PRNGKey(42)

h = 36

n_steps = 100000

use_tiny_dna = True

if use_tiny_dna:
  program = updated_tiny_dna
else:
  ku, key = jr.split(key)
  program = agent_logic.initialize(ku)
ku, key = jr.split(key)
program = mutator.initialize(ku, program)
ku, key = jr.split(key)


# 128 is TOO SMALL!
N_MAX_PROGRAMS = 256

programs = jp.repeat(program[None, :], N_MAX_PROGRAMS, axis=0)

env = create_eruption_env(h, config)
ku, key = jr.split(key)
programs, env = run_eruption_env(
    ku, config, programs, env, agent_logic, mutator, n_steps, zoom_sz=12)


In [ ]:
# If the video is too big, and trying to run it crashes the runtime, perform a
# manual download.
from colabtools import fileedit
fileedit.download_file('video.mp4', ephemeral=True)

### Testing tiny_dna on a regular height (72)

In [ ]:
# first just initialize the environment to a certain state, without making videos.
#key = jr.PRNGKey(42)  # half failure. Some promising mutations but ultimately, fail.
key = jr.PRNGKey(137)

h = 72

n_steps = 50000

use_tiny_dna = True

if use_tiny_dna:
  program = updated_tiny_dna
else:
  ku, key = jr.split(key)
  program = agent_logic.initialize(ku)

ku, key = jr.split(key)
program = mutator.initialize(ku, program)

# 128 is TOO SMALL!
N_MAX_PROGRAMS = 256

programs = jp.repeat(program[None, :], N_MAX_PROGRAMS, axis=0)

env = create_eruption_env(h, config)
ku, key = jr.split(key)
programs, env = run_eruption_env(
    ku, config, programs, env, agent_logic, mutator, n_steps, zoom_sz=6)


In [ ]:
# continue...
n_steps = 50000
ku, key = jr.split(key)
programs, env = run_eruption_env(
    ku, config, programs, env, agent_logic, mutator, n_steps, zoom_sz=6,
    steps_per_frame=64, when_to_double_speed=[])

### Extra: extract the surviving plants and evaluate them

In [ ]:
# Extract a living program from the final environment
def extract_alive_agents_sorted(env, programs):
  aid_flat = env.agent_id_grid.flatten()
  is_agent_flat = config.etd.is_agent_fn(env.type_grid).flatten().astype(jp.float32)
  n_alive_per_id = jax.ops.segment_sum(is_agent_flat, aid_flat, num_segments=programs.shape[0])

  h, w = env.agent_id_grid.shape
  agent_wpos_flat = jp.repeat(jp.arange(w)[None,:], h, axis=0).flatten()
  min_w_per_id = jax.ops.segment_min(
      agent_wpos_flat +
       (1 - is_agent_flat) * w  # if you are not an agent, put it high as a filter.
      , aid_flat, num_segments=programs.shape[0])

  alive_mask = n_alive_per_id>0
  alive_programs = programs[alive_mask]
  alive_minw = min_w_per_id[alive_mask]

  sorted_idx = jp.argsort(alive_minw)
  return alive_programs[sorted_idx], alive_minw[sorted_idx]

alive_programs, alive_minw = extract_alive_agents_sorted(env, programs)
print(alive_programs.shape)

In [ ]:
# test the leftmost alive program
key = jr.PRNGKey(42)
h = 72
w = evm.infer_width(h, "landscape")
test_env = evm.create_default_environment(config, h, w)
test_env = evm.place_seed(test_env, w//2, config)

dna_test = alive_programs[0]
key, ku = jr.split(key)
test_freq_lava(ku, test_env, config, dna_test, agent_logic, mutator)
key, ku = jr.split(key)
test_wave_lava(ku, test_env, config, dna_test, agent_logic, mutator)

In [ ]:
# test the rightmost alive program
key = jr.PRNGKey(42)
h = 72
w = evm.infer_width(h, "landscape")
test_env = evm.create_default_environment(config, h, w)
test_env = evm.place_seed(test_env, w//2, config)

dna_test = alive_programs[-1]
key, ku = jr.split(key)
test_freq_lava(ku, test_env, config, dna_test, agent_logic, mutator)
key, ku = jr.split(key)
test_wave_lava(ku, test_env, config, dna_test, agent_logic, mutator)

### Extra: see what happens after 50k more steps

In [ ]:
# continue...
n_steps = 50000
ku, key = jr.split(key)
next_programs, next_env = run_eruption_env(
    ku, config, programs, env, agent_logic, mutator, n_steps, zoom_sz=6,
    steps_per_frame=64, when_to_double_speed=[])

## Example for a (failing) Petri dish meta-evolution

Using half the width of landscape, so that you can show before/after in the same video.


In [ ]:
# Let's use less tall environments, so that we can scale it later on to be very very wide.
ec_id = "persistence"
env_width_type = "landscape"
h = 72
env_width_type = evm.infer_width(h, "landscape") // 2 # half 16:9
env_and_config = evm.get_env_and_config(ec_id, width_type=env_width_type, h=h,
                                        etd=etd)
st_env, config = env_and_config

w = st_env.type_grid.shape[1]

agent_model = "extended" #@param ['minimal', 'extended']
agent_logic = BasicAgentLogic(config, minimal_net=agent_model=="minimal")

petri_lava_perc = jp.full([w], 0.005)

In [ ]:

def visualize_petri_run(key, programs, agent_logic, st_env, config, n_steps,
                  init_steps_per_frame=2,
                  when_to_double_speed=[100, 500, 1000, 2000, 5000],
                  fps=20,
                  zoom_sz=8):
  env = st_env
  steps_per_frame = init_steps_per_frame

  def make_frame(env):
    return zoom(evm.grab_image_from_env(env, config), zoom_sz)

  frame = make_frame(env)

  out_file = "video.mp4"
  with media.VideoWriter(out_file, shape=frame.shape[:2], fps=fps,
                         crf=18) as video:
    for i in tqdm.trange(n_steps):
      if i in when_to_double_speed:
        steps_per_frame *= 2

      key, k1 = jr.split(key)
      env = update_slice_with_lava(k1, env, 1, 0, petri_lava_perc, etd)
      key, ku = jr.split(key)
      env, n_successful_repr = step_env(
          ku, env, config, agent_logic, programs, do_reproduction=True,
          mutate_programs=False, intercept_reproduction=True,
          min_repr_energy_requirement=jp.zeros([2]), # we don't care about it here.
          excl_fs=excl_fs)

      if i % steps_per_frame == 0:
        video.add_image(make_frame(env))

  media.show_video(media.read_video(out_file))

### Show an example petri run

In [ ]:
# visualize the result in a petri world first.
key = jr.PRNGKey(43)

# How many unique programs (organisms) are allowed in the simulation.
N_MAX_PROGRAMS = 128

n_steps = 2000

# initialize params
ku, key = jr.split(key)
init_program = agent_logic.initialize(ku)
programs = jp.repeat(init_program[None,:], N_MAX_PROGRAMS, axis=0)

ku, key = jr.split(key)
visualize_petri_run(ku, programs, agent_logic, st_env, config, n_steps)


### Meta evolve with petri

In [ ]:
from evojax.algo import PGPE

key = jr.PRNGKey(137)


n_steps = 2000

# initialize params
ku, key = jr.split(key)
init_program = agent_logic.initialize(ku)
# don't use a mutator!

pop_size = 64 # if you run out of memory, decrease this.
ku, key = jr.split(key)
solver = PGPE(
    pop_size=pop_size,
    param_size=init_program.shape[0],
    optimizer='adam',
    center_learning_rate=0.001,
    stdev_learning_rate=0.001,
    stdev_max_change=0.002,
    seed=ku[0],
    init_params=init_program,
    init_stdev=0.001,
)

n_max_programs = 1 # No reproduction occurs, after all.

# how many agents we want in the environment at any point in time.
# For a change, let's aim at a lot here.
n_agent_target = 200
min_repr_energy_requirement = (config.dissipation_per_step * 4) + config.specialize_cost * 2

def count_agents_f(env, etd):
  return etd.is_agent_fn(env.type_grid).sum()


@jit
def fitness_f(key, program):
  def body_f(i, carry):
    key, env, fit = carry

    key, k1 = jr.split(key)
    env = update_slice_with_lava(k1, env, 1, 0, petri_lava_perc, etd)
    key, keyused = jr.split(key)
    new_env, n_successful_repr = step_env(
        keyused, env, config, agent_logic, program[None,:], do_reproduction=True,
          mutate_programs=False, intercept_reproduction=True,
        min_repr_energy_requirement=min_repr_energy_requirement,
        excl_fs=excl_fs)

    n_agent_fit = -jp.abs(n_agent_target - count_agents_f(new_env, config.etd))
    repr_fit = n_successful_repr * n_agent_target * 4

    fit = fit + n_agent_fit + repr_fit
    return key, new_env, fit

  # process the integrity first
  env = process_structural_integrity_n_times(st_env, config, 10)
  _, f_env, fit = jax.lax.fori_loop(0, n_steps, body_f, (key, env, 0.))
  return fit / n_steps

@jit
def v_fitness_f(key, v_params):
  return vmap(fitness_f)(jr.split(key, pop_size), v_params)

mean_fit_log = []
max_fit_log = []

In [ ]:

for _ in range(30):
  # sample
  programs = solver.ask()
  # eval
  key, k1 = jr.split(key)
  fitness = v_fitness_f(k1, programs)
  # update
  solver.tell(fitness)

  mean_fitness = fitness.mean()
  max_fitness = fitness.max()
  mean_fit_log.append(mean_fitness)
  max_fit_log.append(max_fitness)
  print(mean_fitness, max_fitness)
  if len(mean_fit_log) % 10 == 0:
    clear_output()
    plt.plot(mean_fit_log, label="mean_fitness")
    plt.plot(max_fit_log, label="max_fitness")
    plt.legend()
    plt.show()

### meta-evolved petri run

In [ ]:
# visualize the result in a petri world first.
key = jr.PRNGKey(42)

# How many unique programs (organisms) are allowed in the simulation.
N_MAX_PROGRAMS = 128

n_steps = 2000

programs = jp.repeat(solver.best_params[None,:], N_MAX_PROGRAMS, axis=0)

ku, key = jr.split(key)
visualize_petri_run(ku, programs, agent_logic, st_env, config, n_steps)


### Test the result

In [ ]:
mutator_type = "randomly_adaptive" #@param ['basic', 'randomly_adaptive']
# try a smaller sd in general.
sd = 1e-4
mutator = (BasicMutator(sd=sd, change_perc=0.2) if mutator_type == "basic"
           else RandomlyAdaptiveMutator(init_sd=sd, change_perc=0.2))

key = jr.PRNGKey(42)
h = 72
w = evm.infer_width(h, "landscape")
test_env = evm.create_default_environment(config, h, w)
test_env = evm.place_seed(test_env, w//2, config)

dna_test = solver.best_params

key, ku = jr.split(key)
dna_test = mutator.initialize(ku, dna_test)

key, ku = jr.split(key)
test_freq_lava(ku, test_env, config, dna_test, agent_logic, mutator)
key, ku = jr.split(key)
test_wave_lava(ku, test_env, config, dna_test, agent_logic, mutator)